<a href="https://colab.research.google.com/github/Zhou-spec/OTC_Marketing_Making/blob/main/policy_iteration_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class ResidualBlock_Conv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock_Conv, self).__init__()

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)

        out += residual
        out = self.relu(out)

        return out

class ResNet_Conv(nn.Module):
    def __init__(self, intput_size, output_size, input_channels, output_channels, num_blocks, final_act):
        super(ResNet_Conv, self).__init__()

        self.fc = nn.Linear(intput_size, 2096)
        self.conv1 = nn.Conv1d(input_channels, output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.final_act = final_act

        self.blocks = nn.ModuleList()
        for _ in range(num_blocks):
            self.blocks.append(ResidualBlock_Conv(output_channels, output_channels))

        self.conv2 = nn.Conv1d(output_channels, input_channels, kernel_size=3, stride=1, padding=1)
        self.fc2 = nn.Linear(2096, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, output_size)

    def forward(self, t, S, q):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x = torch.tensor([t, S, q], device = device)
        out = self.fc(x)
        out = out.unsqueeze(0)
        out = out.unsqueeze(0)
        out = self.conv1(out)
        out = self.relu(out)

        for block in self.blocks:
            out = block(out)

        out = self.conv2(out)
        out = self.relu(out)
        out = out.squeeze()
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.fc5(out)
        out = self.final_act(out)


        return out

In [ ]:
def Stock_Prices_Simulation(T, dt, sigma, S0):
    # T: the total time needed
    # dt: the time interval
    # sigma: the volatility
    # S0: the initial stock price
    # the output is the simulated stock prices in torch tensor

    # let S be on the same device as S0
    N = int(T / dt)
    device = S0.device
    S = torch.zeros(N, device = device)
    S[0] = S0
    for i in range(1, N):
        S[i] = S[i - 1] + sigma * (torch.sqrt(torch.tensor([dt])) * torch.randn(1)).to(device)

    return S


def Market_Order_Generator(bid_vector, ask_vector, A, B, dt):
    # bid_vector: the bid price vector, torch tensor of size N
    # ask_vector: the ask price vector, torch tensor of size N
    # dt: the time interval
    # In this project, we assume that MO intensity lambda = A - B * epsilon

    N = len(A)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    buy_orders = torch.zeros(N, device = device)
    sell_orders = torch.zeros(N, device = device)

    bid_intensity = A - B * bid_vector
    ask_intensity = A - B * ask_vector
    buy_orders = torch.distributions.poisson.Poisson(bid_intensity * dt).sample()
    sell_orders = torch.distributions.poisson.Poisson(ask_intensity * dt).sample()

    return buy_orders, sell_orders

def initial_Gaussian_Policy(t, S, q, net, A, B, Q, z, delta, gamma):
    N = len(A)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bid_vector = torch.zeros(N, device = device)
    ask_vector = torch.zeros(N, device = device)

    mean = net.forward(t, S, q)
    bid_mean = mean[:int(len(mean) / 2)]
    ask_mean = mean[int(len(mean) / 2):]
    covariance_matrix = torch.diag(gamma / (2 * z * B))
    covariance_matrix = covariance_matrix.to(device)
    bid_vector = torch.distributions.multivariate_normal.MultivariateNormal(bid_mean, covariance_matrix).sample()
    ask_vector = torch.distributions.multivariate_normal.MultivariateNormal(ask_mean, covariance_matrix).sample()

    return bid_vector, ask_vector

def initial_Train_Data_Simulation(T, dt, sigma, S0, A, B, Q, z, delta, gamma, net):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N = int(T / dt)
    S = Stock_Prices_Simulation(T, dt, sigma, S0)
    buy_orders = torch.zeros(N, len(A))
    sell_orders = torch.zeros(N, len(A))
    buy_orders = buy_orders.to(device)
    sell_orders = sell_orders.to(device)
    q = torch.zeros(N, device = device)
    t = torch.zeros(N, device = device)
    bid_vectors = torch.zeros(N, len(A))
    ask_vectors = torch.zeros(N, len(A))
    bid_vectors = bid_vectors.to(device)
    ask_vectors = ask_vectors.to(device)
    for i in range(N - 1):
        bid_vector, ask_vector = initial_Gaussian_Policy(t[i], S[i], q[i], net, A, B, Q, z, delta, gamma)
        bid_vectors[i] = bid_vector
        ask_vectors[i] = ask_vector
        buy_orders[i], sell_orders[i] = Market_Order_Generator(bid_vector, ask_vector, A, B, dt)
        for j in range(len(A)):
            q[i + 1] += (buy_orders[i][j] - sell_orders[i][j]) * z[j]
        q[i + 1] += q[i]
        t[i + 1] = t[i] + dt

    return S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors


def value_function_loss(net, S, q, t, dt):
    N = len(q)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss = torch.zeros(N, device = device)
    for i in range(N - 1):
        loss[i] = (net.forward(t[i + 1], S[i + 1], q[i + 1]) - net.forward(t[i], S[i], q[i])) / dt
    return loss

def inventory_loss(net, S, q, t, dt, buy_orders, sell_orders, z, delta, Q, A, B):
    N = len(q)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss = torch.zeros(N, device = device)
    for i in range(N):
        for k in range(len(A)):
            loss[i] = loss[i] + (buy_orders[i][k] - sell_orders[i][k]) * (z[k] * S[i]) - delta * q[i] * q[i]
            loss[i] = loss[i] +  (A[k] / (2 * B[k]) - (net.forward(t[i], S[i], q[i] + z[k]) - net.forward(t[i], S[i], q[i]) + z[k] * S[i]) / (2 * z[k])) * buy_orders[i][k]
            loss[i] = loss[i] + (A[k] / (2 * B[k]) - (net.forward(t[i], S[i], q[i] - z[k]) - net.forward(t[i], S[i], q[i]) - z[k] * S[i]) / (2 * z[k])) * sell_orders[i][k]
    return loss

def total_loss(net, S, q, t, dt, buy_orders, sell_orders, z, delta, Q, A, B, gamma):
    N = len(S)
    K = len(A)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss = torch.zeros(N, device = device)
    loss1 = value_function_loss(net, S, q, t, dt)
    loss2 = inventory_loss(net, S, q, t, dt, buy_orders, sell_orders, z, delta, Q, A, B)
    loss = loss1 + loss2 - gamma * ((K * 1.7981798683) + torch.sum(gamma / (2 * z * B)))

    scalar_loss = 0.5 * torch.sum(loss[:-1] ** 2) * dt * dt
    return scalar_loss

In [ ]:
T = torch.tensor([1])
dt = torch.tensor([0.01])
S0 = torch.tensor([1])
q = torch.tensor([0])
A = torch.tensor([20, 18, 15, 12, 10, 8])
B = torch.tensor([1, 1, 1, 1, 1, 1])
Q = torch.tensor([300]) # this quantity doesn't matter since I changed the reward function
z = torch.tensor([10, 20, 30, 40, 50, 60])
delta = torch.tensor([0.01]) # this quantity also doesn't matter anymore
gamma = torch.tensor([0.01])
sigma = torch.tensor([0.05])

# use cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
T = T.to(device)
dt = dt.to(device)
S0 = S0.to(device)
q = q.to(device)
A = A.to(device)
B = B.to(device)
Q = Q.to(device)
z = z.to(device)
delta = delta.to(device)
gamma = gamma.to(device)
sigma = sigma.to(device)

In [ ]:
res_cnn_value = ResNet_Conv(3, 1, 1, 2, 1, nn.Identity()).to(device)
res_cnn_policy = ResNet_Conv(3, 12, 1, 2, 1, nn.Sigmoid()).to(device)
optimizer = torch.optim.Adam(res_cnn_value.parameters(), lr = 0.02)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 5, gamma = 0.95)
loss_policy = []
data = []


In [ ]:
for _ in range(50):
  S, buy_orders, sell_orders, q, t, bid_vector, ask_vectors = initial_Train_Data_Simulation(T, dt, sigma, S0, A, B, Q, z, delta, gamma, res_cnn_policy)
  data.append([S, buy_orders, sell_orders, q, t, bid_vector, ask_vectors])

In [ ]:
import numpy as np
from numpy import random
for epoch in range(50):
    loss = 0
    holder = epoch % 5
    S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors = data[random.randint(50)]
    loss = total_loss(res_cnn_value, S, q, t, dt, buy_orders, sell_orders, z, delta, Q, A, B, gamma)
    loss_policy.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
    print('epoch: ', epoch, 'loss: ', loss.item() / 10)

Epoch-0 lr: 0.02
epoch:  0 loss:  75536.51875
Epoch-1 lr: 0.02
epoch:  1 loss:  2183742.0
Epoch-2 lr: 0.02
epoch:  2 loss:  20915.15
Epoch-3 lr: 0.02
epoch:  3 loss:  40011.271875
Epoch-4 lr: 0.019
epoch:  4 loss:  179885.625
Epoch-5 lr: 0.019
epoch:  5 loss:  75566.5
Epoch-6 lr: 0.019
epoch:  6 loss:  297516.725
Epoch-7 lr: 0.019
epoch:  7 loss:  44505.715625
Epoch-8 lr: 0.019
epoch:  8 loss:  39785.634375
Epoch-9 lr: 0.01805
epoch:  9 loss:  27660.484375
Epoch-10 lr: 0.01805
epoch:  10 loss:  295814.275
Epoch-11 lr: 0.01805
epoch:  11 loss:  38913.91875
Epoch-12 lr: 0.01805
epoch:  12 loss:  5258.915234375
Epoch-13 lr: 0.01805
epoch:  13 loss:  43330.565625
Epoch-14 lr: 0.0171475
epoch:  14 loss:  6197.282421875
Epoch-15 lr: 0.0171475
epoch:  15 loss:  3336.372265625
Epoch-16 lr: 0.0171475
epoch:  16 loss:  2409.4267578125
Epoch-17 lr: 0.0171475
epoch:  17 loss:  3883.936328125
Epoch-18 lr: 0.0171475
epoch:  18 loss:  84322.04375
Epoch-19 lr: 0.016290125
epoch:  19 loss:  54619.88125

In [ ]:
def Gaussian_Policy(t, S, q, net, A, B, Q, z, delta, gamma):
    # t: the current time
    # S: the current stock price
    # q: the current inventory
    # net: the neural network
    # This function is used to generate the bid and ask price vectors under Gaussian policy

    N = len(A)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bid_vector = torch.zeros(N, device = device)
    ask_vector = torch.zeros(N, device = device)

    for i in range(N):
        bid_mean = (A[i] / (2 * B[i])) - (net.forward(t, S, q + z[i]) - net.forward(t, S, q) + z[i] * S ) / (2 * z[i])
        ask_mean = (A[i] / (2 * B[i])) - (net.forward(t, S, q - z[i]) - net.forward(t, S, q) - z[i] * S ) / (2 * z[i])
        variance = gamma / (2 * z[i] * B[i])
        std = torch.sqrt(variance)
        bid_vector[i] = torch.normal(bid_mean, std)
        ask_vector[i] = torch.normal(ask_mean, std)

    return bid_vector, ask_vector

def Train_Data_Simulation(T, dt, sigma, S0, A, B, Q, z, delta, gamma, net):
    # T: the total time needed
    # dt: the time interval
    # sigma: the volatility
    # S0: the initial stock price
    # net: the neural network that represent the value function
    # this function return the simulated stock prices, buy orders, sell orders, inventory and time for N time steps

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N = int(T / dt)
    S = Stock_Prices_Simulation(T, dt, sigma, S0)
    buy_orders = torch.zeros(N, len(A))
    sell_orders = torch.zeros(N, len(A))
    buy_orders = buy_orders.to(device)
    sell_orders = sell_orders.to(device)
    q = torch.zeros(N, device = device)
    t = torch.zeros(N, device = device)
    bid_vectors = torch.zeros(N, len(A))
    ask_vectors = torch.zeros(N, len(A))
    bid_vectors = bid_vectors.to(device)
    ask_vectors = ask_vectors.to(device)
    for i in range(N - 1):
        bid_vector, ask_vector = Gaussian_Policy(t[i], S[i], q[i], net, A, B, Q, z, delta, gamma)
        bid_vectors[i] = bid_vector
        ask_vectors[i] = ask_vector
        buy_orders[i], sell_orders[i] = Market_Order_Generator(bid_vector, ask_vector, A, B, dt)
        for j in range(len(A)):
            q[i + 1] += (buy_orders[i][j] - sell_orders[i][j]) * z[j]
        q[i + 1] += q[i]
        t[i + 1] = t[i] + dt

    return S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors

def final_return(S, q, buy_orders, sell_orders, bid_vectors, ask_vectors, T, dt, z):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N = int(T / dt)
    final_return = 0
    for i in range(N):
      final_return += torch.sum(z * buy_orders[i] * bid_vectors[i])
      final_return += torch.sum(z * sell_orders[i] * ask_vectors[i])

    final_return += q[-1] * S[-1]
    return final_return

In [ ]:
value_net = ResNet_Conv(3, 1, 1, 2, 1, nn.Identity()).to(device)
policy_net = res_cnn_value

reward_distribution = []
bid_mat = []
ask_mat = []
for iteration in range(5):

  new_data = []
  r_dis = []
  for _ in range(50):
    S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors = Train_Data_Simulation(T, dt, sigma, S0, A, B, Q, z, delta, gamma, policy_net)
    new_data.append([S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors])
    r = final_return(S, q, buy_orders, sell_orders, bid_vectors, ask_vectors, T, dt, z)
    r_dis.append(r.item())

  reward_distribution.append(r_dis)
  optimizer = torch.optim.Adam(value_net.parameters(), lr = 0.02)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 5, gamma = 0.95)

  for epoch in range(50):
    loss = 0
    S, buy_orders, sell_orders, q, t, bid_vectors, ask_vectors = new_data[random.randint(50)]
    loss = total_loss(value_net, S, q, t, dt, buy_orders, sell_orders, z, delta, Q, A, B, gamma)
    loss_policy.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
    print('epoch: ', epoch, 'loss: ', loss.item() / 10)

  policy_net = value_net
  value_net = ResNet_Conv(3, 1, 1, 2, 1, nn.Identity()).to(device)

Epoch-0 lr: 0.02
epoch:  0 loss:  166134.875
Epoch-1 lr: 0.02
epoch:  1 loss:  774579712.0
Epoch-2 lr: 0.02
epoch:  2 loss:  78514.58125
Epoch-3 lr: 0.02
epoch:  3 loss:  137388.9875
Epoch-4 lr: 0.019
epoch:  4 loss:  117259.55
Epoch-5 lr: 0.019
epoch:  5 loss:  30521.01875
Epoch-6 lr: 0.019
epoch:  6 loss:  18980.9390625
Epoch-7 lr: 0.019
epoch:  7 loss:  67243.2375
Epoch-8 lr: 0.019
epoch:  8 loss:  337520.25
Epoch-9 lr: 0.01805
epoch:  9 loss:  11290.0890625
Epoch-10 lr: 0.01805
epoch:  10 loss:  108307.975
Epoch-11 lr: 0.01805
epoch:  11 loss:  29507.7
Epoch-12 lr: 0.01805
epoch:  12 loss:  602114.65
Epoch-13 lr: 0.01805
epoch:  13 loss:  68205.175
Epoch-14 lr: 0.0171475
epoch:  14 loss:  77853.25625
Epoch-15 lr: 0.0171475
epoch:  15 loss:  78831.11875
Epoch-16 lr: 0.0171475
epoch:  16 loss:  23465.709375
Epoch-17 lr: 0.0171475
epoch:  17 loss:  638986.1
Epoch-18 lr: 0.0171475
epoch:  18 loss:  8091.12109375
Epoch-19 lr: 0.016290125
epoch:  19 loss:  7582.615625
Epoch-20 lr: 0.0162

In [ ]:
t_mat = torch.tensor([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).to(device)
S_mat = torch.tensor([0.9, 0.92, 0.94, 0.96, 0.98, 1, 1.02, 1.04, 1.06, 1.08]).to(device)
q_mat = torch.tensor([50, 100, 150, 200, 250, 300, 350, 400, 450, 500]).to(device)
bid_mat = torch.zeros(10, 10, 10, 6).to(device)
ask_mat = torch.zeros(10, 10, 10, 6).to(device)

def bid_ask_mean(t, S, q, net, A, B, Q, z, delta, gamma):
    # t: the current time
    # S: the current stock price
    # q: the current inventory
    # net: the neural network
    # This function is used to generate the bid and ask price vectors under Gaussian policy

    N = len(A)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bid_vector = torch.zeros(N, device = device)
    ask_vector = torch.zeros(N, device = device)

    for i in range(N):
        bid_mean = (A[i] / (2 * B[i])) - (net.forward(t, S, q + z[i]) - net.forward(t, S, q) + z[i] * S ) / (2 * z[i])
        ask_mean = (A[i] / (2 * B[i])) - (net.forward(t, S, q - z[i]) - net.forward(t, S, q) - z[i] * S ) / (2 * z[i])
        bid_vector[i] = bid_mean
        ask_vector[i] = ask_mean

    return bid_vector, ask_vector

for i in range(1):
  for j in range(1):
    for k in range(1):
      bid_vector, ask_vector = bid_ask_mean(t_mat[i], S_mat[j], q_mat[k], policy_net, A, B, Q, z, delta, gamma)
      bid_mat[i][j][k] = bid_vector
      ask_mat[i][j][k] = ask_vector


print(bid_mat[0][0][0])
print(ask_mat[0][0][0])

print(bid_ask_mean(t_mat[0], S_mat[0], q_mat[0], res_cnn_value, A, B, Q, z, delta, gamma))

tensor([9.5500, 8.5500, 7.0500, 5.5500, 4.5500, 3.5500], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([10.4500,  9.4500,  7.9500,  6.4500,  5.4500,  4.4500], device='cuda:0',
       grad_fn=<SelectBackward0>)
(tensor([9.5500, 8.5500, 7.0500, 5.5500, 4.5500, 3.5500], device='cuda:0',
       grad_fn=<CopySlices>), tensor([10.4500,  9.4500,  7.9500,  6.4500,  5.4500,  4.4500], device='cuda:0',
       grad_fn=<CopySlices>))


In [ ]:
c = np.array(reward_distribution)
d = bid_mat.cpu().detach().numpy()
e = ask_mat.cpu().detach().numpy()
np.save('reward_distribution_policy_iteration.npy', c)
np.save('bid_mat_policy_iteration.npy', d)
np.save('ask_mat_policy_iteration.npy', e)